# 01. Box Office 데이터 수집

### 0. OPEN API의 이해

In [ ]:
# RESTful API를 문서화, 공유 위한 표준 명세
    # RESTful API에선 주로 GET요청 (SELECT, 검색) 기능만 공개함
    # 주로 JSON형식의 요청과 응답 주고받음

# OPEN API -> 외부데이터 -> 서버(컴터) from:
    # 공공데이터 포털
    # 기업 내부 API

# 공공데이터들을 규격화, 정제, 가공, 분석을 위해서는 python 작업이 필수!!
    # -> openAPI로 제공되는 데이터 구조 화긴 후 코딩 ㄱㄱ

### 1. OpenAPI 명세 확인하기

In [ ]:
# chrome 창에서 응답구조 확인:
# 요청 url:
    # ex. 영화진흥위원회:
    # http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json
# 요청변수 및 변수값
    # 각 사이트의 변수명, 값 규격 맞춰서 적기
    # 요청변수의 default 값도 잘 보자!!
        # ex. if default= 전체 
            # => 굳이 함수나 변수로 안만들어도 되거나 신경 안써도 되는 것들임
    # ex. 영화진흥위원회:
    # 발급받은키, 조회날짜
# 요청 (GET (SELECT) 방식)
# -> https://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f7417133af2db5f416098ffc27436891&targetDt=20251104


### 2. 라이브러리 참조하기 
###### (VScode로 넘어와서 작업 ㄱ)

In [2]:
import requests
import datetime as dt
from pandas import DataFrame

### 3. 요청정보 확인 (a.k.a. 변수 설정)

In [3]:
# API 명세를 확인 -> 요청에 필요한 정보들 변수로 구성 ㄱ

# 요청 URL:
url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json"

# 발급받은 API 키:
API_KEY = "f7417133af2db5f416098ffc27436891"

# 하루 전 날짜 얻기 (당일건 집계 전이라 안됨):
date = dt.datetime.now() - dt.timedelta(days=1)
yesterday = date.strftime("%Y%m%d")
yesterday

'20251104'

### 4. 웹사이트에 데이터 요청하기

In [6]:
# 사이트에서 데이터를 규격에 맞게 요청, 응답 받는다
    # GET 방식 요청
        # -> QueryString은 params 파라미터에 딕셔너리 형식으로 구성됨
        # (POST, PUT, DELETE는 data로 파라미터 이름 지정)
with requests.Session() as session:
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    r = session.get(url, params={"key": API_KEY, "targetDt": yesterday})

    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    
    print(r)

<Response [200]>


### 5. 응답결과 화긴

In [8]:
mydict = r.json()
print(mydict)

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스', 'showRange': '20251104~20251104', 'dailyBoxOfficeList': [{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20253852', 'movieNm': '퍼스트 라이드', 'openDt': '2025-10-29', 'salesAmt': '215330100', 'salesShare': '28.4', 'salesInten': '-15052260', 'salesChange': '-6.5', 'salesAcc': '3827605550', 'audiCnt': '23991', 'audiInten': '-1187', 'audiChange': '-4.7', 'audiAcc': '417973', 'scrnCnt': '1016', 'showCnt': '3437'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20256757', 'movieNm': '극장판 체인소 맨: 레제편', 'openDt': '2025-09-24', 'salesAmt': '145836700', 'salesShare': '19.2', 'salesInten': '-15110280', 'salesChange': '-9.4', 'salesAcc': '29873971080', 'audiCnt': '14654', 'audiInten': '-1471', 'audiChange': '-9.1', 'audiAcc': '2824910', 'scrnCnt': '630', 'showCnt': '2218'}, {'rnum': '3', 'rank': '3', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20257570', 'movieNm': '8번 출구', 'op

### 6. 응답결과를 프레임화

In [10]:
df = DataFrame(mydict["boxOfficeResult"]["dailyBoxOfficeList"])
df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20253852,퍼스트 라이드,2025-10-29,215330100,28.4,-15052260,-6.5,3827605550,23991,-1187,-4.7,417973,1016,3437
1,2,2,0,OLD,20256757,극장판 체인소 맨: 레제편,2025-09-24,145836700,19.2,-15110280,-9.4,29873971080,14654,-1471,-9.1,2824910,630,2218
2,3,3,0,OLD,20257570,8번 출구,2025-10-22,73737100,9.7,-1595300,-2.1,3312283070,7719,57,0.7,338381,572,1219
3,4,4,1,OLD,20247429,세계의 주인,2025-10-22,33099400,4.4,-12060,0,691881630,3508,54,1.6,72589,286,412
4,5,5,-1,OLD,20090284,코렐라인,2009-05-21,34029000,4.5,-2344000,-6.4,2947228900,3485,-214,-5.8,375694,208,463
5,6,6,0,OLD,20243561,어쩔수가없다,2025-09-24,25282360,3.3,-1647360,-6.1,28601174150,2639,-227,-7.9,2925421,397,554
6,7,7,0,OLD,20253289,극장판 귀멸의 칼날: 무한성편,2025-08-22,24021300,3.2,-114700,-0.5,60210835630,2439,-31,-1.3,5579215,341,440
7,8,8,0,OLD,20228988,보스,2025-10-03,17947700,2.4,-1472300,-7.6,23283571050,2003,-155,-7.2,2424810,344,463
8,9,9,1,OLD,20225855,하얀 차를 탄 여자,2025-10-29,17127900,2.3,4743700,38.3,162430480,1747,418,31.5,17563,304,400
9,10,10,-1,OLD,20256397,원 배틀 애프터 어나더,2025-10-01,15017500,2.0,879000,6.2,5539934800,1551,73,4.9,521265,189,246


# 2. 카카오 검색결과 수집(1)

In [ ]:
# POST 방식의 요청
# -> 파라미터 이름: data, 응답구조: Thunder Client로 화긴

# 요즘 인증키 보내는 방식 트랜드는 header에 포함하는 거임
    # 보안이슈에 더 좋대... 뭔말일까나

## 카카오 개발자 센터
###### https://developers.kakao.com/

In [ ]:
# https://developers.kakao.com/
    # 카카오가 제공하는 OpenAPI 연동신청 및 
    # API 요청,응답 규격 확인하기 위한 사이트

# 카카오는 key 이름을 앱이라고 함

# lookup for: 검색API 명세 화긴
    # for 코딩에 필요한 
        # : 인증방식, 요청규격, 리스트 포함위치, 응답구조, 응답규격 
        # if 요청방법 안내가 다소 불친절 or 배경지식 요구
        # -> thunder client로 요청 ㄱㄱ

    # thunder client를 쓴다 해도 요청규격 및 리스트 포함위치, 응답받은 변수의 의미를 파악하기 위해서는
    # 코딩 전에 사이트의 명세 확인은 필수이다!!

### 1. 카카오 검색 API 명세 확인

In [18]:
# 발급받은 REST API 키: 
    # e4ad8ce76913bf44a6532c21e51e82ab
# API 전송방법 화긴: 
    # QueryString 방식 x -> header 전송방식
# 요청정보 화긴:
    # QueryString 방식 -> GET 방식
        # inferred from: '쿼리 파라미터' -> 이름: query, 타입: String
    # 2500 data we can get from one query
        # inferred from: page x size (size는 페이지 당 정보 수)
# 응답정보의 리스트 위치 화긴:
    # 이름: documents
        # dic 안의 list임 -> 표로 변환 가능 -> 열(field) 정보 화긴
# if: 요청방법 안내가 다소 불친절 or 배경지식 요구
    # -> thunder client로 요청 ㄱㄱ
    # GET
    # url: https://dapi.kakao.com/v3/search/book
    # Query
        # 요청변수: query, page, size
    # Headers
        # API key: e4ad8ce76913bf44a6532c21e51e82ab
        # Accept: */*
        # User-Agent: Thunder Client
        # Authorization: KakaoAK 인증키

### 2. 라이브러리 참조하기

In [ ]:
# 위의 영화진흥위원회 import들이랑 중복 되도 상관 ㄴ
import requests
from pandas import DataFrame

### 3. 요청정보 화긴

In [24]:
# 사이트에서 확인한 API 명세 규격 참고해서
# Resrful API에 전달할 변수들 정의

# 요청 URL
url = "https://dapi.kakao.com/v3/search/book"

# QueryString 요청변수
query = "파이썬"
page = 1
size = 50

# headers에 포함할 키
key = "e4ad8ce76913bf44a6532c21e51e82ab"

### 4. 웹 데이터 요청하기

In [25]:
# headers에 인증key 추가 + QueryString 파라미터 구성하기
with requests.Session() as session:
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Authorization": "KakaoAK %s" % key})

    r = session.get(url, params={"query": query, "page": page, "size": size})

    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    
    print(r)

<Response [200]>


### 5. 응답결과 json으로 변환, 구조 화긴

In [ ]:
mydict = r.json()
print(mydict)
# -> 딕셔너리를 포함하는 리스트의 시작 위치를 파악해야 됨! 
    # for 프레임화
        # 이번 결과에서는 'document'가 딕셔너리를 포함하는 리스트의 시작 위치임

{'documents': [{'authors': ['박응용'], 'contents': '개정 2판으로 새롭게 태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 설명과 예제를 다듬고, 최신 경향과 심화 내용을 보충했다. 또한 이번 개정 2판도 50만 코딩 유튜버인 조코딩과 협업을 통해 유튜브 동영상을 제공해 파이썬을 더 쉽게 공부할 수 있다.  8년 연속 베스트셀러! 위키독스 누적 방문 300만! 독자의 입에서 입으로 전해진 추천과 수많은 대학 및 학원의 교재 채택을 통해 검증은 이미 끝났다. 코딩을 처음 배우는 중고등학생부터 코딩', 'datetime': '2023-06-15T00:00:00.000+09:00', 'isbn': '1163034738 9791163034735', 'price': 22000, 'publisher': '이지스퍼블리싱', 'sale_price': 19800, 'status': '정상판매', 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F6361131%3Ftimestamp%3D20250709143830', 'title': 'Do it! 점프 투 파이썬', 'translators': [], 'url': 'https://search.daum.net/search?w=bookpage&bookId=6361131&q=Do+it%21+%EC%A0%90%ED%94%84+%ED%88%AC+%ED%8C%8C%EC%9D%B4%EC%8D%AC'}, {'authors': ['윤인성'], 'contents': '『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었습니다. 프로그래밍이 정말 처음인 입문자도 따라갈 수 있는 친절한 설명과 단계별 학습은 그대로! 혼자 공부하더라도 체계적으로 계획을 세워 학습할 수 있도록 ‘혼공 계획표’를 새롭게 추가했습니다. 또한 입문자가 자주 물어보는 질

### 6. 응답결과를 프레임화 & 엑셀 저장하기

In [ ]:
# 수집데이터를 재사용해야 된다면 엑셀파일로 저장해야됨
df = DataFrame(mydict["documents"])
df.to_excel("카카오 책 검색(%s)).xlsx" % query)

### 7. 저장된 엑셀은 VScode Excel 익스텐션으로 화긴 ㄱ
###### 생성된 파일 클릭 후 office viewer 선택하면 열림

# 03. 카카오 검색결과 수집(2)

###### 페이지 단위로 데이터 화긴하기

###### 데이터 요청을 반복문으로 처리하기

###### 연속된 데이터를 수집하기 위한 계획 수립

In [ ]:
# page 번호만 바꾸면 다른 페이지들 추가가능!!!
    # 반복문 돌릴 수 있음을 암시
        # -> for문으로 get함수 감싸기

# 반복문 결과가 저장되야됨 
    # -> 빈 리스트 생성 
    # 반복문 + 리스트 크기 확장문인 .extend() 를 쓴다면
        # 수집한 '총 데이터 수'에 대한 정보도 알 수 있음!
            # len(resultset)

# if error 뜸 + raise Exception 처리
    # -> 에러 뜨기 전 결과들 다 지워짐!
        # -> raise Exceprtion 지우고
        # continue 추가 필요 암시

### 1. 라이브러리 참조하기

In [28]:
# 사용할 라이브러리 import 해오기
import requests
from pandas import DataFrame

### 2. 요청정보 화긴

In [29]:
# 직전 카카오 책검색 실습과 동일

# 요청 URL
url = "https://dapi.kakao.com/v3/search/book"

# QueryString 요청변수
query = "파이썬"
page = 1
size = 50

# headers에 포함할 키
key = "e4ad8ce76913bf44a6532c21e51e82ab"

### 3. 웹데이터 요청하기

In [31]:
# 요청 결과를 저장하기 위한 빈 리스트 생성
resultset = []

with requests.Session() as session:
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Authorization": "KakaoAK %s" % key})

    for i in range(1, 51):  # i: page
        r = session.get(url, params={"query": query, "page": i, "size": size})

        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            # for: 중간에 에러 발생해서 stop 하는거 방지
            #raise Exception(msg)
            print(msg) #raise Exception 지우고 에러 뜬거 확인용으로 print + continue 넣기
            continue

        print("%d page 데이터 수신 완료 >> %s" %(i, r)) # r: 페이지당 response[200]
        mydict = r.json()
        resultset.extend(mydict['documents'])


print("총 %d건의 데이터가 수집되었습니다." % len(resultset))

1 page 데이터 수신 완료 >> <Response [200]>
2 page 데이터 수신 완료 >> <Response [200]>
3 page 데이터 수신 완료 >> <Response [200]>
4 page 데이터 수신 완료 >> <Response [200]>
5 page 데이터 수신 완료 >> <Response [200]>
6 page 데이터 수신 완료 >> <Response [200]>
7 page 데이터 수신 완료 >> <Response [200]>
8 page 데이터 수신 완료 >> <Response [200]>
9 page 데이터 수신 완료 >> <Response [200]>
10 page 데이터 수신 완료 >> <Response [200]>
11 page 데이터 수신 완료 >> <Response [200]>
12 page 데이터 수신 완료 >> <Response [200]>
13 page 데이터 수신 완료 >> <Response [200]>
14 page 데이터 수신 완료 >> <Response [200]>
15 page 데이터 수신 완료 >> <Response [200]>
16 page 데이터 수신 완료 >> <Response [200]>
17 page 데이터 수신 완료 >> <Response [200]>
18 page 데이터 수신 완료 >> <Response [200]>
19 page 데이터 수신 완료 >> <Response [200]>
20 page 데이터 수신 완료 >> <Response [200]>
21 page 데이터 수신 완료 >> <Response [200]>
22 page 데이터 수신 완료 >> <Response [200]>
23 page 데이터 수신 완료 >> <Response [200]>
24 page 데이터 수신 완료 >> <Response [200]>
25 page 데이터 수신 완료 >> <Response [200]>
26 page 데이터 수신 완료 >> <Response [200]>
27 page 데이터 수신 완료 >> 

### 4. 데이터 저장하기

In [32]:
# 수집된 데이터 재사용 위해 엑셀파일로 저장
df = DataFrame(resultset) # 아까는 mydict["documents"] 였음
df.to_excel("카카오 책 검색(%s)_전체 데이터).xlsx" % query)

### 5. 수집된 데이터 중 임의의 한 건 확인하기

In [ ]:
# 축소판 이미지(thumbnail)
    # 표지 이미지
    # 원본이미지 보다 작으니까 파일 용량도 작음
    # 다운로드 모드: stream=True

# 표지 이미지 다운받아야됨
    # 동기식 처리 되면..ㅎ -> 비동기 처리 ㄱㄱ

# -> 데이터 하나 출력해서 thumbnail 저장위치 화긴하기
print(resultset[0])

# 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F6361131%3Ftimestamp%3D20250709143830'

{'authors': ['박응용'], 'contents': '개정 2판으로 새롭게 태어났다! 챗GPT를 시작으로 펼쳐진 생성 AI 시대에 맞춰 설명과 예제를 다듬고, 최신 경향과 심화 내용을 보충했다. 또한 이번 개정 2판도 50만 코딩 유튜버인 조코딩과 협업을 통해 유튜브 동영상을 제공해 파이썬을 더 쉽게 공부할 수 있다.  8년 연속 베스트셀러! 위키독스 누적 방문 300만! 독자의 입에서 입으로 전해진 추천과 수많은 대학 및 학원의 교재 채택을 통해 검증은 이미 끝났다. 코딩을 처음 배우는 중고등학생부터 코딩', 'datetime': '2023-06-15T00:00:00.000+09:00', 'isbn': '1163034738 9791163034735', 'price': 22000, 'publisher': '이지스퍼블리싱', 'sale_price': 19800, 'status': '정상판매', 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F6361131%3Ftimestamp%3D20250709143830', 'title': 'Do it! 점프 투 파이썬', 'translators': [], 'url': 'https://search.daum.net/search?w=bookpage&bookId=6361131&q=Do+it%21+%EC%A0%90%ED%94%84+%ED%88%AC+%ED%8C%8C%EC%9D%B4%EC%8D%AC'}


### 6. 표지이미지 파일 다운로드 함수 정의

In [36]:
def download(url, target):
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    try:
        r = session.get(url, stream=True)
        r.encoding = "utf-8"

        with open(target, "wb") as f:
            f.write(r.raw.read())
            print(target, "가 저장 되었습니다.")

    except Exception as e:
        print(target, "저장실패", e)

### 7. 반복문으로 수집했던 결과 중 표지이미지 파일 다운로드 받기

In [38]:
# 동기식 처리 (순차적 다운로드)
    # 다운로드 시간 확인 필요
import os
import datetime as dt

# 다운로드 결과를 저장할 폴더 생성하기
dirname = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
os.mkdir(dirname)

# 수집된 데이터 내에서 이미지 파일 추출
    # 이미지 다운로드용 반복문 ㄱㄱ
for i, v in enumerate(resultset):
    # 저장될 파일경로 문자열 생성
    file_path = os.path.join(dirname, "%d.jpg" % i)
    # 다운로드 ㄱㄱ
    download(v['thumbnail'], file_path)

20251105-163300\0.jpg 가 저장 되었습니다.
20251105-163300\1.jpg 가 저장 되었습니다.
20251105-163300\2.jpg 가 저장 되었습니다.
20251105-163300\3.jpg 가 저장 되었습니다.
20251105-163300\4.jpg 가 저장 되었습니다.
20251105-163300\5.jpg 가 저장 되었습니다.
20251105-163300\6.jpg 가 저장 되었습니다.
20251105-163300\7.jpg 가 저장 되었습니다.
20251105-163300\8.jpg 가 저장 되었습니다.
20251105-163300\9.jpg 가 저장 되었습니다.
20251105-163300\10.jpg 가 저장 되었습니다.
20251105-163300\11.jpg 가 저장 되었습니다.
20251105-163300\12.jpg 가 저장 되었습니다.
20251105-163300\13.jpg 가 저장 되었습니다.
20251105-163300\14.jpg 가 저장 되었습니다.
20251105-163300\15.jpg 가 저장 되었습니다.
20251105-163300\16.jpg 가 저장 되었습니다.
20251105-163300\17.jpg 가 저장 되었습니다.
20251105-163300\18.jpg 가 저장 되었습니다.
20251105-163300\19.jpg 가 저장 되었습니다.
20251105-163300\20.jpg 가 저장 되었습니다.
20251105-163300\21.jpg 가 저장 되었습니다.
20251105-163300\22.jpg 가 저장 되었습니다.
20251105-163300\23.jpg 가 저장 되었습니다.
20251105-163300\24.jpg 가 저장 되었습니다.
20251105-163300\25.jpg 가 저장 되었습니다.
20251105-163300\26.jpg 가 저장 되었습니다.
20251105-163300\27.jpg 가 저장 되었습니다.
20251105-163300\28.jpg 가 저장 되었

In [41]:
# 비동기식 처리 (한번에 다운로드)
    # 다운로드 시간 불필요
from concurrent import futures

# 다운로드 결과를 저장할 폴더 생성하기
dirname = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
os.mkdir(dirname)

# 수집된 데이터 내에서 비동기 작업으로 이미지 파일 추출
with futures.ThreadPoolExecutor() as executor:
    # 이미지 다운로드용 반복문 ㄱㄱ:
    for i, v in enumerate(resultset):
        # 저장될 파일경로 문자열 생성:
        file_path = os.path.join(dirname, "%d.jpg" % i)
        # 비동기식 다운로드 요청 ㄱ
        executor.submit(download, v['thumbnail'], file_path)

20251105-163803\1.jpg 가 저장 되었습니다.
20251105-163803\7.jpg 가 저장 되었습니다.
20251105-163803\5.jpg 가 저장 되었습니다.
20251105-163803\16.jpg 가 저장 되었습니다.
20251105-163803\17.jpg 가 저장 되었습니다.
20251105-163803\9.jpg 가 저장 되었습니다.
20251105-163803\15.jpg 가 저장 되었습니다.
20251105-163803\10.jpg 가 저장 되었습니다.
20251105-163803\18.jpg 가 저장 되었습니다.
20251105-163803\20.jpg 가 저장 되었습니다.
20251105-163803\21.jpg 가 저장 되었습니다.
20251105-163803\22.jpg 가 저장 되었습니다.
20251105-163803\19.jpg 가 저장 되었습니다.
20251105-163803\23.jpg 가 저장 되었습니다.
20251105-163803\24.jpg 가 저장 되었습니다.
20251105-163803\27.jpg 가 저장 되었습니다.
20251105-163803\25.jpg 가 저장 되었습니다.
20251105-163803\28.jpg 가 저장 되었습니다.
20251105-163803\29.jpg 가 저장 되었습니다.
20251105-163803\26.jpg 가 저장 되었습니다.
20251105-163803\30.jpg 가 저장 되었습니다.
20251105-163803\31.jpg 가 저장 되었습니다.
20251105-163803\32.jpg 가 저장 되었습니다.
20251105-163803\33.jpg 가 저장 되었습니다.
20251105-163803\37.jpg 가 저장 되었습니다.
20251105-163803\35.jpg 가 저장 되었습니다.
20251105-163803\34.jpg 가 저장 되었습니다.
20251105-163803\36.jpg 가 저장 되었습니다.
20251105-163803\38.jpg 가

In [42]:
print(file_path)

20251105-163803\2499.jpg


In [43]:
print(dirname)

20251105-163803


# 04. 네이버 트렌드 테이터수집

In [ ]:
# https://developers.naver.com/main/
# id: 29alsdud / pw: Anna5703!
# RestAPI KEY (두개 다 복사ㄱ):
    # client id: KB20VIIlMJQIa8_EWzuq
    # client Secret: QKASGiawb2
# 요청 url: https://openapi.naver.com/v1/datalab/search

### 0. 연동규격 확인하기

In [ ]:
# 목표: '리스트가 들어있는 위지' 찾기

# JSON 형식 -> 딕셔너리라는 뜻
    # array -> 배열이라는 뜻, 즉 리스트!
    # keywordGroups: array(JSON),
        # keywordGroups.keywords: array(string)
            
# 전체 구조: 딕 ⊃ 리슽 ⊃ 딕 ⊃ 리스트!!

# API 요청 시 HTTP 요청 
    # headers에 client id, client secret 둘 다 추가:
        # X-Naver-Client-Id: 
        # X-Naver-Client-Secret:

### 1. 라이브러리 참조하기

In [ ]:
# 요청 파라미터를 json 형식으로 전달하기 위해
# json 라이브러리 추가 ㄱㄱ
import requests
import json
import datetime as dt
from pandas import DataFrame

### 2. 요청정보 확인

In [ ]:
# 검색하고싶은거 내가 다른거 검색해도 괜춘하대

# 요청 url:
url = "https://openapi.naver.com/v1/datalab/search"

# 네이버 인증키:
clientId: "KB20VIIlMJQIa8_EWzuq"
clientSecret: "QKASGiawb2"

# 날짜변수 생성 (오늘날짜, 1년전 날짜)
now = dt.datetime.now()     # 검색시작일
today = now.strftime("%Y-%m-%d")    # 검색종료일
startDateDelta = now - dt.timedelta(days=365)
startDate = startDateDelta.strftime("%Y-%m-%d")

data = {
"startDate": startDate,     # 검색시작일
    "endDate": today,       # 검색종료일
    "timeUnit": "date",     # date, week, month
    "keywordGroups": [
        {
            "groupName": "Benz",    # 검색주제
            # 주제와 관련된 검색어:
            "keywords": ["벤츠", "e클래스", "s클래스", "c클래스", "벤츠 cls"]
        },
        {
            "groupName": "BMW"
            "keywords":  ["BMW", "5시리즈", "x5", "x7", "x3"]
        },
        {
            "groupName": "AUDI"
            "keywords":  ["아우디", "a6", "e트론", "a7", "q5"]
        }
    ]
}

### 3. 웹에 데이터 요청하기

In [ ]:
# data=json.dumps(data)
    # 딕셔너리 -> 문자열로 변환

### 4. 응답결과 확인

### 5. 응답결과 재 구성

### 6. 최종_Data_Set -> 프레임화